## Summary of data
## Properties from San Jose, CA
## 6099 condos
## 92 different neighborhoods

In [1]:
PROPERTY_TYPES = ['condo', 'townhouse', 'multiunit', 'singlefamily']
TRESH = 0.15
NUM_BINS = 5
BINS = []

MONTHS = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 
          'Jun', 'Jul', 'Aug', 'Sep', 'Oct',
         'Nov', 'Dec']

COLS_EXCLUDE = ['Median Sale Price','Region','neighborhood',
                'location','city','property_type','state',
                'Month of Period End', 'Median_Sale_Price_Bin', 'neighborhood_num',
                'Month of Period End']

In [29]:
import csv
import itertools
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path

import pyzipcode as pz
from uszipcode import ZipcodeSearchEngine

In [3]:
# convert string percentages to numerical
def convert_percentage(df):
    for col in df.columns:
        c = col.split(" ")
        if c[-1] == "MoM" or c[-1] == "YoY": 
            df[col] = [float(str(x).strip("%")) for x in df[col]]
    
    df['Average Sale To List'] = [float(str(x).strip("%")) for x in df['Average Sale To List']]

In [4]:
### read in data
def read_data(filename, property_type):
    df = pd.read_csv(filename)
    df.columns = [x.strip(" ") for x in df.columns]
    df['property_type'] = property_type
    df['property_type_num'] = PROPERTY_TYPES.index(property_type)
    df.head()
    return (df)

In [5]:
### add columns for city, state, and neighborhood
def parse_region(df):
    df['location'] = [x.split(",") for x in df.Region]
    df['city'] = [x[0].strip(",").lower() for x in df.location]
    df['state'] = [x[1].split(" ")[1].lower().strip() for x in df.location]
    df['neighborhood'] = [x[1].split(" ")[3].strip().lower() for x in df.location]
    df = df[df.city == 'san jose']
    df.head()

In [6]:
### convert Median Sale Price to a float
def convert_med_sale_pr(df):
    df[u'Median Sale Price'] = [x.strip("$").replace(",", "") for x in df["Median Sale Price"]]
    df[u'Median Sale Price'] = [int(x) if x[len(x)-1] != "K" else int(x.strip("K"))*1000 
                                for x in df["Median Sale Price"]]

In [7]:
### convert neighborhood to a numerical value
def convert_neighborhood(df):
    neighborhoods = df.neighborhood.unique()
    df['neighborhood_num'] = [np.where(neighborhoods == x)[0][0] for x in df.neighborhood]

In [8]:
# encode catgorial variable so that it can be used
# in regression
def encode_categorial(df):
    neighborhoods = df.neighborhood.unique()
    for n in neighborhoods:
        df[n] = [1 if x == n else 0 for x in df.neighborhood]
    return (df)

In [9]:
# take median sale prices, bin, and add a column 
# that represents price ranges
def bin_med_sale_pr(df, bins):
    med_sale_pr = df['Median Sale Price']
    bins = np.linspace(med_sale_pr.min(), med_sale_pr.max(), bins)
    digitized = np.digitize(med_sale_pr, bins)
    df['Median_Sale_Price_Bin'] = digitized
    return (df, bins)

In [10]:
# Take date and turn it into month and year
def parse_date(df):
    df['Month of Period End'] = [x.replace('-', ' ') for x in df['Month of Period End']]
    df['Month'] = [MONTHS.index(x.split(' ')[0][:3]) for x in df['Month of Period End']]
    df['Year'] = [int(x.split(' ')[1]) for x in df['Month of Period End']]
    return (df)

In [11]:
def filter_prices(df, high, low):
    t_cutoff = np.percentile(sorted(df['Median Sale Price']), high)
    b_cutoff = np.percentile(sorted(df['Median Sale Price']), low)
    df = df[df['Median Sale Price'] <= t_cutoff]
    df = df[df['Median Sale Price'] >= b_cutoff]
    return (df)

In [41]:
def latlong_from_zip(df):
    search = ZipcodeSearchEngine()
    df['long'] = [search.by_zipcode(str(int(x)))['Longitude'] for x in df['Zip Code']]
    df['lat'] = [search.by_zipcode(str(int(x)))['Latitude'] for x in df['Zip Code']]
    return (df)

In [42]:
### given the filename and property type return
### a dataframe with string fields converted to
### numerical and regions parsed out
def clean_data(filename, property_type):
    df = read_data(filename, property_type)
    df = df[df['Median Sale Price'].isnull() == False]
    convert_med_sale_pr(df)
    parse_region(df)
    convert_percentage(df)
    convert_neighborhood(df)
    # df = parse_date(df)
    df = latlong_from_zip(df)
    df = filter_prices(df, 75, 10)
    return (df)

In [43]:
# merge several dataframes together
# files = list of tuples, 
# each tuple should contain filename first then property type

# How to deal with NA's? fill with 0 for now ...
def merge_data(files):
    frames = []
    for f in files:
        frames.append(clean_data(f[0], f[1]))
    
    df = pd.concat(frames).fillna(method='ffill')
    return (df)

In [44]:
files = [('sanjose_condos_1.csv', PROPERTY_TYPES[0]), ('sanjose_townhouse2_1.csv', PROPERTY_TYPES[1]), 
 ('sanjose_multiunit_1.csv', PROPERTY_TYPES[2]), ("sanjose_singlefamily_1.csv", PROPERTY_TYPES[3])]

total = merge_data(files)
total, bins = bin_med_sale_pr(total, NUM_BINS)

In [45]:
total.head()

,Region,Month of Period End,Median Sale Price,Median Sale Price MoM,Median Sale Price YoY,Homes Sold,Homes Sold MoM,Homes Sold YoY,New Listings,New Listings MoM,...,property_type,property_type_num,location,city,state,neighborhood,neighborhood_num,long,lat,Median_Sale_Price_Bin
41,"San Jose, CA - Alexander",15-Jun,352000,6.3,30.6,3.0,-25.0,-25.0,9.0,-10.0,...,condo,0,"[San Jose, CA - Alexander]",san jose,ca,alexander,0,-121.850568,37.355863,1
49,"San Jose, CA - Alexander",16-Feb,365000,8.1,29.2,13.0,-7.1,116.7,6.0,-33.3,...,condo,0,"[San Jose, CA - Alexander]",san jose,ca,alexander,0,-121.850568,37.355863,1
50,"San Jose, CA - Alexander",16-Mar,355000,-2.7,20.4,6.0,-53.8,100.0,7.0,16.7,...,condo,0,"[San Jose, CA - Alexander]",san jose,ca,alexander,0,-121.850568,37.355863,1
51,"San Jose, CA - Alexander",16-Apr,365000,2.7,17.7,3.0,-50.0,-40.0,6.0,-14.3,...,condo,0,"[San Jose, CA - Alexander]",san jose,ca,alexander,0,-121.850568,37.355863,1
52,"San Jose, CA - Alexander",16-May,346000,-5.3,4.4,5.0,66.7,25.0,8.0,33.3,...,condo,0,"[San Jose, CA - Alexander]",san jose,ca,alexander,0,-121.850568,37.355863,1


In [46]:
total.to_csv('san_jose_properties_zipcodes.csv')